In [1]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import numpy as np
from tqdm import tqdm,trange
import matplotlib.pyplot as plt
import numba

In [2]:
infile = "data/signal/wza_UL18.root" # --signal

#infile = "data/mc/TTZToLL/30FD3469-AEFB-B742-8F3E-4551B23D529B.root"
#infile='data/data/7B56E217-555E-1C41-9494-491849A9835F_skim_2ElIdPt20.root' # --data 
#infile = 'data/data/CA42F3A2-614F-4A4F-AF18-F6E66CDA2C85_skim_2ElIdPt20.root'
#infile = 'data/Ntuple/EGamma_Run2018B.root'
#infile="data/mc/59AB328B-F0E3-F544-98BB-E5E55577C649_skim_2ElIdPt20.root" # --mc


year='2018'

In [3]:
events = NanoEventsFactory.from_root(infile, schemaclass=NanoAODSchema).events()

In [4]:
# Trigger set
doubleelectron_triggers  ={
    '2018': [
            "Ele23_Ele12_CaloIdL_TrackIdL_IsoVL", # Recomended
            ]
}



singleelectron_triggers = { #2017 and 2018 from monojet, applying dedicated trigger weights
        '2016': [
            'Ele27_WPTight_Gsf',
            'Ele105_CaloIdVT_GsfTrkIdT'
        ],
        '2017': [
            'Ele35_WPTight_Gsf',
            'Ele115_CaloIdVT_GsfTrkIdT',
            'Photon200'
        ],
        '2018': [
            'Ele32_WPTight_Gsf',    # Recomended
        ]
    }

In [5]:
# << flat dim helper function >>
def flat_dim(arr):

#   if len(ak.to_numpy(arr).shape) > 1:
#       sub_arr = ak.flatten(arr)
#   else:
#       sub_arr = arr
    sub_arr = ak.flatten(arr)
    mask = ~ak.is_none(sub_arr)

    return ak.to_numpy(sub_arr[mask])
# << drop na helper function >>
def drop_na(arr):

    mask = ~ak.is_none(arr)

    return arr[mask]
# << drop na helper function >>
def drop_na_np(arr):

    mask = ~np.isnan(arr)

    return arr[mask]
# << Sort by PT  helper function >>
def sort_by_pt(ele,pho,jet):
    ele = ele[ak.argsort(ele.pt,ascending=False,axis=1)]
    pho = pho[ak.argsort(pho.pt,ascending=False,axis=1)]
    jet = jet[ak.argsort(jet.pt,ascending=False,axis=1)]

    return ele,pho,jet

# << Delta R helper function >>
def delta_r(eta1, eta2, phi1, phi2):
    deta = abs(eta1 - eta2)
    dphi = abs(np.mod(phi1 - phi2 + np.pi, 2*np.pi) - np.pi)
    dr = np.sqrt(deta**2 + dphi**2)
    return deta, dphi, dr

### 1. Trigger

In [6]:
# double lepton trigger
is_double_ele_trigger=True
if not is_double_ele_trigger:
    double_ele_triggers_arr=np.ones(len(events), dtype=np.bool)
else:
    double_ele_triggers_arr = np.zeros(len(events), dtype=np.bool)
    for path in doubleelectron_triggers[year]:
        if path not in events.HLT.fields: continue
        double_ele_triggers_arr = double_ele_triggers_arr | events.HLT[path]


# single lepton trigger
is_single_ele_trigger=True
if not is_single_ele_trigger:
    single_ele_triggers_arr=np.ones(len(events), dtype=np.bool)
else:
    single_ele_triggers_arr = np.zeros(len(events), dtype=np.bool)
    for path in singleelectron_triggers[year]:
        if path not in events.HLT.fields: continue
        single_ele_triggers_arr = single_ele_triggers_arr | events.HLT[path]

# Sort particle order by PT  # RunD --> has problem
events.Electron,events.Photon,events.Jet = sort_by_pt(events.Electron,events.Photon,events.Jet)

Initial_events = events
#events = events[single_ele_triggers_arr | double_ele_triggers_arr]
events = events[double_ele_triggers_arr]
print("Events passing triggers and skiimig: ",len(events) )

cut1 = np.ones(len(events))
# Particle Identification


Events passing triggers and skiimig:  9544


In [7]:
Electron = events.Electron
Photon = events.Photon
Jet = events.Jet
MET = events.MET
Muon = events.Muon

In [8]:
isData = 'genWeight' not in events.fields

### 2. Lepton Selection: e e e

In [9]:
# Muon only used to calculate dR
MuSelmask = (Muon.pt > 10) & (abs(Muon.eta) < 2.5)  & (Muon.tightId) & (Muon.pfRelIso04_all < 0.15)
#Muon = ak.mask(Muon,MuSelmask)
Muon = Muon[MuSelmask]

In [10]:
Muon

<MuonArray [[], [], [], [], ... [], [], [], []] type='9544 * var * muon'>

In [11]:
# Electron selection
EleSelmask = ((Electron.pt > 10) & (np.abs(Electron.eta + Electron.deltaEtaSC) < 1.479)  &  (Electron.cutBased > 2) & (abs(Electron.dxy) < 0.05) & (abs(Electron.dz) < 0.1)) | \
            ((Electron.pt > 10) & (np.abs(Electron.eta + Electron.deltaEtaSC) > 1.479) & (np.abs(Electron.eta + Electron.deltaEtaSC) < 2.5) & (Electron.cutBased > 2) & (abs(Electron.dxy) < 0.1) & (abs(Electron.dz) < 0.2))

Electron = Electron[EleSelmask]

In [12]:
Electron

<ElectronArray [[Electron], ... Electron], []] type='9544 * var * electron'>

In [13]:
Tri_electron_mask = ak.num(Electron) == 3

In [14]:
Electron = Electron[Tri_electron_mask]
Photon = Photon[Tri_electron_mask]
Jet = Jet[Tri_electron_mask]
MET = MET[Tri_electron_mask]
Muon = Muon[Tri_electron_mask]
print("Events with 3 selected electrons: ",len(Photon))

Events with 3 selected electrons:  918


### 3. Photon Selection:  pho > 0 

In [15]:
# Photon selection

isgap_mask = (abs(Photon.eta) < 1.442)  |  ((abs(Photon.eta) > 1.566) & (abs(Photon.eta) < 2.5))
Pixel_seed_mask = ~Photon.pixelSeed
PT_ID_mask = (Photon.pt > 20) & (Photon.cutBased > 1)

dr_pho_ele = ak.all(Photon.metric_table(Electron) > 0.5, axis=-1) # default metric table: delta_r
dr_pho_mu = ak.all(Photon.metric_table(Muon) > 0.5, axis=-1)


#if not isData:
#    isPrompt = (Photon.genPartFlav == 1) | (Photon.genPartFlav == 11)
#    PhoSelmask = (pho.pt > 20) & (pho.cutBased > 1) & isgap_mask &  Pixel_seed_mask & isPrompt
#if isData:
#    PhoSelmask = (pho.pt > 20) & (pho.cutBased > 1) & isgap_mask &  Pixel_seed_mask 

PhoSelmask = (Photon.pt > 20) & (Photon.cutBased > 1) & isgap_mask &  Pixel_seed_mask & dr_pho_ele & dr_pho_mu

Photon = Photon[PhoSelmask]

In [16]:
A_photon_mask = ak.num(Photon) > 0

In [17]:
Electron = Electron[A_photon_mask ]
Photon   = Photon[A_photon_mask]
Jet = Jet[A_photon_mask]
Muon = Muon[A_photon_mask]
MET = MET[A_photon_mask]

In [18]:
print("Eventes with 3 ele and 1 or more pho",len(Photon))

Eventes with 3 ele and 1 or more pho 262


### OSSF Selection:  Ze1 Ze2 We1

In [19]:
##-----------  Cut flow3: Electron Selection --> OSSF 
# OSSF index maker
@numba.njit
def find_3lep(events_leptons,builder):
    for leptons in events_leptons:

        builder.begin_list()
        nlep = len(leptons)
        for i0 in range(nlep):
            for i1 in range(i0+1,nlep):
                if leptons[i0].charge + leptons[i1].charge != 0: continue;

                for i2 in range(nlep):
                    if len({i0,i1,i2}) < 3: continue;
                    builder.begin_tuple(3)
                    builder.index(0).integer(i0)
                    builder.index(1).integer(i1)
                    builder.index(2).integer(i2)
                    builder.end_tuple()
        builder.end_list()
    return builder

eee_triplet_idx = find_3lep(Electron,ak.ArrayBuilder()).snapshot()

# OSSF cut
ossf_mask = ak.num(eee_triplet_idx) == 2
eee_triplet_idx = eee_triplet_idx[ossf_mask]

Electron= Electron[ossf_mask]
Photon= Photon[ossf_mask]
Jet= Jet[ossf_mask]
MET = MET[ossf_mask]

/home/jkim/miniconda3/envs/pyHEP/lib/python3.8/site-packages/numba/core/dispatcher.py:238: UserWarning: Numba extension module 'awkward1._connect._numba' failed to load due to 'ImportError(generic_type: type "kernel_lib" is already registered!)'.
  entrypoints.init_all()


In [20]:
Triple_electron = [Electron[eee_triplet_idx[idx]] for idx in "012"]
from coffea.nanoevents.methods import vector
ak.behavior.update(vector.behavior)

def TLorentz_vector(vec):
    vec = ak.zip(
    {
                "x":vec.x,
                "y":vec.y,
                "z":vec.z,
                "t":vec.t
    },
    with_name = "LorentzVector"
    )
    return vec

def TLorentz_vector_cylinder(vec):

    vec = ak.zip(
    {
         "pt": vec.pt,
         "eta": vec.eta,
         "phi": vec.phi,
         "mass": vec.mass,
    },
    with_name="PtEtaPhiMLorentzVector",
    )

    return vec



Triple_eee = ak.zip({"lep1":Triple_electron[0],
                            "lep2":Triple_electron[1],
                         "lep3":Triple_electron[2],
                         "p4":TLorentz_vector(Triple_electron[0]+Triple_electron[1])})


In [21]:
bestZ_idx = ak.singletons(ak.argmin(abs(Triple_eee.p4.mass - 91.1876), axis=1))
Triple_eee = Triple_eee[bestZ_idx]
leading_ele, subleading_ele, Third_ele = ak.flatten(TLorentz_vector_cylinder(Triple_eee.lep1)),ak.flatten(TLorentz_vector_cylinder(Triple_eee.lep2)),ak.flatten(TLorentz_vector_cylinder(Triple_eee.lep3))

In [22]:
def make_leading_pair(target,base):
    return target[ak.argmax(base.pt,axis=1,keepdims=True)]

leading_pho     = make_leading_pair(Photon,Photon)

In [23]:
print("OSSF selection: ",len(Triple_eee))

OSSF selection:  261


### Baseline Selection: PT( e e e ), Z Mass, MET

In [24]:
# Z mass window
diele             = Triple_eee.p4
zmass_window_mask = ak.firsts(diele.mass) > 4  

# M(eea) cuts 
#eeg_vec           = diele + leading_pho
#Meeg_mask         = ak.firsts(eeg_vec.mass > 120)

# Electron PT cuts
Elept_mask = ak.firsts((Triple_eee.lep1.pt > 25) & (Triple_eee.lep2.pt > 10) & (Triple_eee.lep3.pt > 25))

# MET cuts
MET_mask = MET.pt > 20

# Mask
Event_sel_mask   = zmass_window_mask  & Elept_mask & MET_mask 

# Apply cyts
Triple_eee_sel   = Triple_eee[Event_sel_mask]
leading_pho_sel   = leading_pho[Event_sel_mask]
# Photon  EE and EB
isEE_mask = leading_pho.isScEtaEE
isEB_mask = leading_pho.isScEtaEB
Pho_EE = leading_pho[isEE_mask & Event_sel_mask]
Pho_EB = leading_pho[isEB_mask & Event_sel_mask]
MET_sel           = MET[Event_sel_mask]

In [25]:
print("Event selection: ",len(Triple_eee_sel))

Event selection:  181
